In [2]:
import requests, re
from bs4 import BeautifulSoup
import json
import pandas as pd

In [3]:
user_agent_pool=[ # User-Agent池
    # Cent Browser 4.3.9.248，Chromium 86.0.4240.198
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36', # 2021.01
    # Cent Browser 5.0.1002.295，Chromium 102.0.5005.167
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 Safari/537.36', # 2022.12
 
    # Edge
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36 Edg/93.0.961.38', # 2021.09
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36 Edg/93.0.961.44',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36 Edg/93.0.961.47',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 Edg/93.0.961.52',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36 Edg/94.0.992.31',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36 Edg/94.0.992.37',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36 Edg/94.0.992.38',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36 Edg/94.0.992.47', # 2021.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36 Edg/94.0.992.50',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36 Edg/95.0.1020.30',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36 Edg/95.0.1020.40',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.44', # 2021.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36 Edg/95.0.1020.53',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 Edg/96.0.1054.29',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.34',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.41', # 2021.12
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36 Edg/96.0.1054.53',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 Edg/96.0.1054.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55', # 2022.01
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 Edg/97.0.1072.69',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36 Edg/97.0.1072.76',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36 Edg/98.0.1108.43', # 2022.02
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36 Edg/98.0.1108.50',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edg/98.0.1108.55',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edg/98.0.1108.56',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edg/98.0.1108.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 Edg/99.0.1150.30', # 2022.03
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 Edg/99.0.1150.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 Edg/99.0.1150.39',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.46',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.52',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36 Edg/99.0.1150.55',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36 Edg/100.0.1185.29', # 2022.04
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36 Edg/100.0.1185.39',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.44',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36 Edg/101.0.1210.32',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36 Edg/101.0.1210.39', # 2022.05
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.47',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30', # 2022.06
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.39',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.124 Safari/537.36 Edg/102.0.1245.41',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.124 Safari/537.36 Edg/102.0.1245.44',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36 Edg/103.0.1264.37',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.66 Safari/537.36 Edg/103.0.1264.44', # 2022.07
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.49',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.71',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.47', # 2022.08
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.63',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36 Edg/104.0.1293.70',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27', # 2022.09
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.33',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.42',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.50',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.53',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.34', # 2022.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.37',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.42',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.52',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.24',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.26',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.35', # 2022.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.52',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.56',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46', # 2022.12
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.76', # 2023.01
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.55',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.61',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.70',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 Edg/109.0.1518.78', # 2023.02
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.41',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.46',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.49',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.50',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.56',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.57',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63', # 2023.03
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.69',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.41',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.43',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.44',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.51',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.34', # 2023.04
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.39',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.46',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.58',
 
    # Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36', # 2021.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36', # 2021.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36', # 2021.12
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36', # 2022.01
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36', # 2022.02
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36', # 2022.03
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.82 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36', # 2022.04
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36', # 2022.05
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.115 Safari/537.36', # 2022.06
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.66 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36', # 2022.07
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36', # 2022.08
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.102 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.54 Safari/537.36', # 2022.09
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.102 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.127 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.5249.91 Safari/537.36', # 2022.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.5249.103 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.5249.119 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.63 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.88 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.106 Safari/537.36', # 2022.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.107 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.5304.122 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.72 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.95 Safari/537.36', # 2022.12
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.99 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.100 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.5359.125 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.75 Safari/537.36', # 2023.01
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.120 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.78 Safari/537.36', # 2023.02
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.104 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.105 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.178 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.180 Safari/537.36', # 2023.03
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.64 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.65 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.111 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.112 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.5563.147 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.50 Safari/537.36', # 2023.04
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.121 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.5615.138 Safari/537.36',
 
    # Chrome Beta
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.41 Safari/537.36', # 2021.09
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.17 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.32 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.40 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.49 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.18 Safari/537.36', # 2021.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.27 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.35 Safari/537.36', # 2021.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36',
 
    # Firefox
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0', # 2021.09
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0', # 2021.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0', # 2021.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0', # 2021.12
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0', # 2022.01
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0', # 2022.02
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0', # 2022.03
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:99.0) Gecko/20100101 Firefox/99.0', # 2022.04
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0', # 2022.05
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0', # 2022.06
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0', # 2022.06
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:103.0) Gecko/20100101 Firefox/103.0', # 2022.07
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0', # 2022.08
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0', # 2022.09
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0', # 2022.10
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:107.0) Gecko/20100101 Firefox/107.0', # 2022.11
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:108.0) Gecko/20100101 Firefox/108.0', # 2022.12
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0', # 2023.01
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0', # 2023.02
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0', # 2023.03
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/112.0', # 2023.04
]
 
import random
 


# 爬虫函数

In [4]:
import time
def get_dict(url):
    time.sleep(0.5)
    headers = {'accept': 'application/json, text/plain, */*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'zh-CN,zh;q=0.9',
'cookie':'''_T_WM=34812304544; WEIBOCN_FROM=1110006030; __bid_n=187b8dbddb8fa1287a4207; FPTOKEN=VYXmO/jhda9+sgAHkMqaAKC+GmdAFsbxw5VJowWTFPS+mLi90XQlH/wv/7DhmN0OhnL4TnjmdUB39i+cqlUtjXY9SZqulq6XYtWHy6+xRZCvxiRCfblpvjpav2ydRO+IjYd43Pa+s+iOthIFAfkFQQlVuxk4CbgPKPGYLUz4YlEFaPJk9td0orCyDgmx0F8AWNkM7QlFPZDHkS5B6Jqx/+0WJsy/1L//40WdQlF0FJfvDLxyNIzdNQ0V93jE9eIoeT4e+UD80wQFNxoe9ikQ0rdWKrzOWj3jjKJ5vVOB0yQdvyHChXGNnwWm9Ew2zJyibO/0tqUaHnD5mQ51hxRqo4j4TixZ8/yGSOCsEaV76SgOxpOIik3X0CZlI9HdVioeA6pdIaAZbeuy6TKMO0ZS/g==|yhwBUYkYzSGi4VT73EqlqZxeB0HP7pQm1/Lh+KpLveA=|10|974c2ce98b1240c1681dfbb679ecaeeb; SUB=_2A25JQ5jyDeRhGeNP4lUV-CfMzzSIHXVqzzi6rDV6PUJbktAGLRH6kW1NTkU8YiiUbA-2uOuh7q6-XuNqwA5LiBnT; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5sXvVTPdl95v2mDK9FcLel5NHD95QfeK.NShn4ehBRWs4Dqc_xi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--fi-i8i-zNi--fi-z7iK.X; SSOLoginState=1682434210; MLOGIN=1; XSRF-TOKEN=e8eef1; mweibo_short_token=c325ae1207; M_WEIBOCN_PARAMS=oid=4892900406593198&lfid=4892900406593198&luicode=20000174&uicode=20000061&fid=4892900406593198''',
'mweibo-pwa': '1',
'referer': 'https://m.weibo.cn/detail/4892900406593198',
'x-requested-with': 'XMLHttpRequest'}
    headers['User-Agent']=random.choice(user_agent_pool)
    d = requests.get(url, headers = headers).json()
    return d

In [85]:
data1 = [] #”铁链女“词条
for page in range(2,59):
    url = '''https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%E9%93%81%E9%93%BE%E5%A5%B3&page_type=searchall&page='''+str(page)
    d = get_dict(url)
    for i in range(len(d['data']['cards'])):
        d1 = d['data']['cards'][i]['card_group'][0]['mblog']
        row = dict()
        row['comments_count']= d1['comments_count']
        row['attitudes_count'] = d1['attitudes_count']
        row['reposts_count'] = d1['reposts_count']
        row['created_at']=d1['created_at']
        if 'status_city' in d1.keys():
            row['status_city'] = d1['status_city']
        else:
            row['status_city']=0
        row['status_country']=d1['status_country']
        if 'status_province' in d1.keys():
            row['status_province']=d1['status_province']
        else:
            row['status_province']=0
        if 'page_info' in d1.keys():
            row['tag']=d1['page_info']['page_title']
        else:
            row['tag']=0    
        row['text']=d1['text']
        row['user_id']=d1['user']['id']
        row['isLongText']=d1['isLongText']
        row['user_screen_name']=d1['user']['screen_name']
        row['user_verified']=d1['user']['verified']
        row['user_gender']=d1['user']['gender']
        row['user_follow_count']=d1['user']['follow_count']
        row['user_followers_count']=d1['user']['followers_count']
        if 'verified_reason' in d1['user'].keys():
            row['user_verified_reason']=d1['user']['verified_reason']
        else:
            row['user_verified_reason']=0
        data1.append(row)

In [1]:
import pandas as pd


In [73]:
df1 = pd.DataFrame(data=data1)
df1.to_excel('df1.xlsx')

In [89]:
df1 = pd.read_excel('./df1.xlsx')

In [10]:
import re
from bs4 import BeautifulSoup

In [32]:
long_text_url = df1[df1['isLongText']==True]['text'].apply(lambda x: 'http://m.weibo.cn'+BeautifulSoup(re.search('''<a +href="\/status\/\d+">''',x).group(), "html.parser").a['href'])

In [51]:
df1_longText = []
for u in list(long_text_url):
    url = re.sub('\/\d+','es/extend?id='+re.search('\d+',u).group(),u)
    d = get_dict(url)
    txt = d['data']['longTextContent']
    df1_longText.append(txt)
df1_longText = pd.DataFrame(data={'text':df1_longText},index=long_text_url.index)

In [90]:
df1.update(df1_longText)

In [92]:
df1.to_excel('merge_df1.xlsx')

In [1]:
import pandas as pd
df1 = pd.read_excel('merge_df1.xlsx')

In [59]:
import re
from bs4 import BeautifulSoup
text_list = list(df1.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a\s*href=".+#<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    texts.append(text)


In [55]:
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)

In [77]:
df11=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})

In [80]:
df1 = pd.concat([df1,df11],axis=1)

In [94]:
user_followers_count_ = []

for i in list(df1['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    else:
        user_followers_count_.append(int(i))
df1['user_followers_count']=user_followers_count_

In [ ]:
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df1 = df1[col_name]
df1.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

In [101]:
df1.to_excel('./df12.xlsx')

# #丰县八孩案一审宣判#话题

In [17]:
import json
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
cookies_str = '''XSRF-TOKEN=613a38; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4890343525059448%26luicode%3D10000011%26lfid%3D100103type%253D1%2526q%253D%2523%25E6%259F%25AF%25E5%2593%2580%25E4%25BA%25B2%25E4%25BA%2586%26fid%3D100103type%253D1%2526q%253D%2523%25E6%259F%25AF%25E5%2593%2580%25E4%25BA%25B2%25E4%25BA%2586%26uicode%3D10000011; mweibo_short_token=8521b27d0a; FPTOKEN=Etyil2NXk6yOECtn8J0gaDBSijvrDoHfSgONECEY83xyDRQyMP/J2EOiYP9KWA8QFjK5eykRcX94KuAXBemE1IelLbrxL7Fd9kJ68HRdkU5zBTSyggeE3fvMmdyot/LKYUbO0kvEMn1UYNo/tTRB7d2z4IClxAxOzzThKy5cgMSNbDv0PyH6d5Bjkj+pLKhXteKbcMp/yj4E0RUF93dxRNRUuXizi2FY/bZ1DyR9Auj9pg3t2YOyKNbKIKMj1DRxqxpesFB6TmRDCkDrH43XbIqjLCahfFVUInCwk0U/dRGpov/J3V2RVRHGEu8mSd+2iwnx6/StdrjFC647ybZSAcvYFXfOK0eWsrnhr3N1zzOwsdPTUm/RdtWYq9YQBcrOAm9psgV26GQ8ggUY3P1RCw==|xRwQ9IsjbgAidNdVbUZitkK/yXLOQq006xI2r97Ja2I=|10|1e2ca4984b0762b3582bd379cd312253; __bid_n=1877020ec7b3c710a04207; WEIBOCN_FROM=1110006030; _T_WM=70256803919; SUB=_2A25JBU46DeRhGeNP4lUV-CfMzzSIHXVqBlJyrDV6PUJbktANLXLQkW1NTkU8YgqvKbZG9CrHm265sDLNMhJkEgtB; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5sXvVTPdl95v2mDK9FcLel5NHD95QfeK.NShn4ehBRWs4Dqc_xi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--fi-i8i-zNi--fi-z7iK.X'''
headers = {'User-Agent':'"Mozilla/5.0 (Windows; U; Windows NT 5.2) Gecko/2008070208 Firefox/3.0.1"'}
cookies = dict(item.strip().split('=',maxsplit=1) for item in cookies_str.split(';'))
data2 = [] ##丰县八孩案一审宣判#话题
for page in range(2,54):
    url = '''https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%23%E4%B8%B0%E5%8E%BF%E5%85%AB%E5%AD%A9%E6%A1%88%E4%B8%80%E5%AE%A1%E5%AE%A3%E5%88%A4%23&page_type=searchall&page='''+str(page)
    d = get_dict(url)
    for i in range(len(d['data']['cards'])):
        d1 = d['data']['cards'][i]['card_group'][0]['mblog']
        row = dict()
        row['comments_count']= d1['comments_count']
        row['attitudes_count'] = d1['attitudes_count']
        row['reposts_count'] = d1['reposts_count']
        row['created_at']=d1['created_at']
        if 'status_city' in d1.keys():
            row['status_city'] = d1['status_city']
        else:
            row['status_city']=0
        row['status_country']=d1['status_country']
        if 'status_province' in d1.keys():
            row['status_province']=d1['status_province']
        else:
            row['status_province']=0  
        row['text']=d1['text']
        row['user_id']=d1['user']['id']
        row['id']=d1['id']
        row['isLongText']=d1['isLongText']
        row['user_screen_name']=d1['user']['screen_name']
        row['user_verified']=d1['user']['verified']
        row['user_gender']=d1['user']['gender']
        row['user_follow_count']=d1['user']['follow_count']
        row['user_followers_count']=d1['user']['followers_count']
        if 'verified_reason' in d1['user'].keys():
            row['user_verified_reason']=d1['user']['verified_reason']
        else:
            row['user_verified_reason']=0
        data2.append(row)
df2 = pd.DataFrame(data=data2)
long_text_url = df2[df2['isLongText']==True]['text'].apply(lambda x: 'http://m.weibo.cn'+BeautifulSoup(re.search('''<a +href="\/status\/\d+">''',x).group(), "html.parser").a['href'])
df2_longText = []
for u in list(long_text_url):
    url = re.sub('\/\d+','es/extend?id='+re.search('\d+',u).group(),u)
    d = get_dict(url)
    txt = d['data']['longTextContent']
    df2_longText.append(txt)
df2_longText = pd.DataFrame(data={'text':df2_longText},index=long_text_url.index)
df2.update(df2_longText)
text_list = list(df2.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df21=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df2 = pd.concat([df2,df21],axis=1)
user_followers_count_ = []

for i in list(df2['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df2['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df2 = df2[col_name]
df2.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

/Users/chiajd/miniconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [18]:
df2.to_excel('./df2.xlsx')

# ”铁链女“词条

In [19]:
import json
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
cookies_str = '''XSRF-TOKEN=613a38; MLOGIN=1; M_WEIBOCN_PARAMS=oid%3D4890343525059448%26luicode%3D10000011%26lfid%3D100103type%253D1%2526q%253D%2523%25E6%259F%25AF%25E5%2593%2580%25E4%25BA%25B2%25E4%25BA%2586%26fid%3D100103type%253D1%2526q%253D%2523%25E6%259F%25AF%25E5%2593%2580%25E4%25BA%25B2%25E4%25BA%2586%26uicode%3D10000011; mweibo_short_token=8521b27d0a; FPTOKEN=Etyil2NXk6yOECtn8J0gaDBSijvrDoHfSgONECEY83xyDRQyMP/J2EOiYP9KWA8QFjK5eykRcX94KuAXBemE1IelLbrxL7Fd9kJ68HRdkU5zBTSyggeE3fvMmdyot/LKYUbO0kvEMn1UYNo/tTRB7d2z4IClxAxOzzThKy5cgMSNbDv0PyH6d5Bjkj+pLKhXteKbcMp/yj4E0RUF93dxRNRUuXizi2FY/bZ1DyR9Auj9pg3t2YOyKNbKIKMj1DRxqxpesFB6TmRDCkDrH43XbIqjLCahfFVUInCwk0U/dRGpov/J3V2RVRHGEu8mSd+2iwnx6/StdrjFC647ybZSAcvYFXfOK0eWsrnhr3N1zzOwsdPTUm/RdtWYq9YQBcrOAm9psgV26GQ8ggUY3P1RCw==|xRwQ9IsjbgAidNdVbUZitkK/yXLOQq006xI2r97Ja2I=|10|1e2ca4984b0762b3582bd379cd312253; __bid_n=1877020ec7b3c710a04207; WEIBOCN_FROM=1110006030; _T_WM=70256803919; SUB=_2A25JBU46DeRhGeNP4lUV-CfMzzSIHXVqBlJyrDV6PUJbktANLXLQkW1NTkU8YgqvKbZG9CrHm265sDLNMhJkEgtB; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5sXvVTPdl95v2mDK9FcLel5NHD95QfeK.NShn4ehBRWs4Dqc_xi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--fi-i8i-zNi--fi-z7iK.X'''
headers = {'User-Agent':'"Mozilla/5.0 (Windows; U; Windows NT 5.2) Gecko/2008070208 Firefox/3.0.1"'}
cookies = dict(item.strip().split('=',maxsplit=1) for item in cookies_str.split(';'))
data1 = [] #”铁链女“词条
for page in range(2,59):
    url = '''https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%E9%93%81%E9%93%BE%E5%A5%B3&page_type=searchall&page='''+str(page)
    d = get_dict(url)
    for i in range(len(d['data']['cards'])):
        d1 = d['data']['cards'][i]['card_group'][0]['mblog']
        row = dict()
        row['comments_count']= d1['comments_count']
        row['attitudes_count'] = d1['attitudes_count']
        row['reposts_count'] = d1['reposts_count']
        row['created_at']=d1['created_at']
        if 'status_city' in d1.keys():
            row['status_city'] = d1['status_city']
        else:
            row['status_city']=0
        row['status_country']=d1['status_country']
        if 'status_province' in d1.keys():
            row['status_province']=d1['status_province']
        else:
            row['status_province']=0  
        row['text']=d1['text']
        row['user_id']=d1['user']['id']
        row['id']=d1['id']
        row['isLongText']=d1['isLongText']
        row['user_screen_name']=d1['user']['screen_name']
        row['user_verified']=d1['user']['verified']
        row['user_gender']=d1['user']['gender']
        row['user_follow_count']=d1['user']['follow_count']
        row['user_followers_count']=d1['user']['followers_count']
        if 'verified_reason' in d1['user'].keys():
            row['user_verified_reason']=d1['user']['verified_reason']
        else:
            row['user_verified_reason']=0
        data1.append(row)
df1 = pd.DataFrame(data=data1)
long_text_url = df1[df1['isLongText']==True]['text'].apply(lambda x: 'http://m.weibo.cn'+BeautifulSoup(re.search('''<a +href="\/status\/\d+">''',x).group(), "html.parser").a['href'])
df1_longText = []
for u in list(long_text_url):
    url = re.sub('\/\d+','es/extend?id='+re.search('\d+',u).group(),u)
    d = get_dict(url)
    txt = d['data']['longTextContent']
    df1_longText.append(txt)
df1_longText = pd.DataFrame(data={'text':df1_longText},index=long_text_url.index)
df1.update(df1_longText)
text_list = list(df1.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df11=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df1 = pd.concat([df1,df11],axis=1)
user_followers_count_ = []

for i in list(df1['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df1['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df1 = df1[col_name]
df1.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id','id', 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

/Users/chiajd/miniconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [20]:
df1.to_excel('./df1.xlsx')

# #董志民获刑9年#

In [21]:
headers = {'User-Agent':'"Mozilla/5.0 (Windows; U; Windows NT 5.2) Gecko/2008070208 Firefox/3.0.1"'}
cookies = dict(item.strip().split('=',maxsplit=1) for item in cookies_str.split(';'))
data3 = [] ##董志民获刑9年#
for page in range(2,4):
    url = '''https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%E8%91%A3%E5%BF%97%E6%B0%91%E8%8E%B7%E5%88%919%E5%B9%B4&page_type=searchall&page='''+str(page)
    d = get_dict(url)
    for i in range(len(d['data']['cards'])):
        d1 = d['data']['cards'][i]['card_group'][0]['mblog']
        row = dict()
        row['comments_count']= d1['comments_count']
        row['attitudes_count'] = d1['attitudes_count']
        row['reposts_count'] = d1['reposts_count']
        row['created_at']=d1['created_at']
        if 'status_city' in d1.keys():
            row['status_city'] = d1['status_city']
        else:
            row['status_city']=0
        row['status_country']=d1['status_country']
        if 'status_province' in d1.keys():
            row['status_province']=d1['status_province']
        else:
            row['status_province']=0  
        row['text']=d1['text']
        row['user_id']=d1['user']['id']
        row['id']=d1['id']
        row['isLongText']=d1['isLongText']
        row['user_screen_name']=d1['user']['screen_name']
        row['user_verified']=d1['user']['verified']
        row['user_gender']=d1['user']['gender']
        row['user_follow_count']=d1['user']['follow_count']
        row['user_followers_count']=d1['user']['followers_count']
        if 'verified_reason' in d1['user'].keys():
            row['user_verified_reason']=d1['user']['verified_reason']
        else:
            row['user_verified_reason']=0
        data3.append(row)
df3 = pd.DataFrame(data=data3)
long_text_url = df3[df3['isLongText']==True]['text'].apply(lambda x: 'http://m.weibo.cn'+BeautifulSoup(re.search('''<a +href="\/status\/\d+">''',x).group(), "html.parser").a['href'])
df3_longText = []
for u in list(long_text_url):
    url = re.sub('\/\d+','es/extend?id='+re.search('\d+',u).group(),u)
    d = get_dict(url)
    txt = d['data']['longTextContent']
    df3_longText.append(txt)
df3_longText = pd.DataFrame(data={'text':df3_longText},index=long_text_url.index)
df3.update(df3_longText)
text_list = list(df3.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df31=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df3 = pd.concat([df3,df31],axis=1)
user_followers_count_ = []

for i in list(df3['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df3['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df3 = df3[col_name]
df3.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id','isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

In [22]:
df3.to_excel('./df3.xlsx')

# #徐州通报丰县生育八孩女子处理情况#

In [23]:
headers = {'User-Agent':'"Mozilla/5.0 (Windows; U; Windows NT 5.2) Gecko/2008070208 Firefox/3.0.1"'}
cookies = dict(item.strip().split('=',maxsplit=1) for item in cookies_str.split(';'))
data4 = [] ##徐州通报丰县生育八孩女子处理情况#
for page in range(2,30):
    url = '''https://m.weibo.cn/api/container/getIndex?containerid=100103type%3D1%26q%3D%23%E5%BE%90%E5%B7%9E%E9%80%9A%E6%8A%A5%E4%B8%B0%E5%8E%BF%E7%94%9F%E8%82%B2%E5%85%AB%E5%AD%A9%E5%A5%B3%E5%AD%90%E5%A4%84%E7%90%86%E6%83%85%E5%86%B5%23&page_type=searchall&page='''+str(page)
    d = get_dict(url)
    for i in range(len(d['data']['cards'])):
        d1 = d['data']['cards'][i]['card_group'][0]['mblog']
        row = dict()
        row['comments_count']= d1['comments_count']
        row['attitudes_count'] = d1['attitudes_count']
        row['reposts_count'] = d1['reposts_count']
        row['created_at']=d1['created_at']
        if 'status_city' in d1.keys():
            row['status_city'] = d1['status_city']
        else:
            row['status_city']=0
        if 'status_country' in d1.keys():
            row['status_country'] = d1['status_country']
        else:
            row['status_country']=0
        if 'status_province' in d1.keys():
            row['status_province']=d1['status_province']
        else:
            row['status_province']=0  
        row['text']=d1['text']
        row['user_id']=d1['user']['id']
        row['isLongText']=d1['isLongText']
        row['id']=d1['id']
        row['user_screen_name']=d1['user']['screen_name']
        row['user_verified']=d1['user']['verified']
        row['user_gender']=d1['user']['gender']
        row['user_follow_count']=d1['user']['follow_count']
        row['user_followers_count']=d1['user']['followers_count']
        if 'verified_reason' in d1['user'].keys():
            row['user_verified_reason']=d1['user']['verified_reason']
        else:
            row['user_verified_reason']=0
        data4.append(row)
df4 = pd.DataFrame(data=data4)
long_text_url = df4[df4['isLongText']==True]['text'].apply(lambda x: 'http://m.weibo.cn'+BeautifulSoup(re.search('''<a +href="\/status\/\d+">''',x).group(), "html.parser").a['href'])
df4_longText = []
for u in list(long_text_url):
    url = re.sub('\/\d+','es/extend?id='+re.search('\d+',u).group(),u)
    d = get_dict(url)
    txt = d['data']['longTextContent']
    df4_longText.append(txt)
df4_longText = pd.DataFrame(data={'text':df4_longText},index=long_text_url.index)
df4.update(df4_longText)
text_list = list(df4.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df41=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df4 = pd.concat([df4,df41],axis=1)
user_followers_count_ = []

for i in list(df4['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df4['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df4 = df4[col_name]
df4.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id','id','isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

In [24]:
df4.to_excel('./df4.xlsx')

[{'comments_count': 0,
  'attitudes_count': 4,
  'reposts_count': 0,
  'created_at': 'Sat Apr 08 09:54:17 +0800 2023',
  'status_city': 0,
  'status_country': '加拿大',
  'status_province': '加拿大',
  'text': '垃圾地方 <a  href="https://m.weibo.cn/search?containerid=231522type%3D1%26t%3D10%26q%3D%23%E5%BE%90%E5%B7%9E%E9%80%9A%E6%8A%A5%E4%B8%B0%E5%8E%BF%E7%94%9F%E8%82%B2%E5%85%AB%E5%AD%A9%E5%A5%B3%E5%AD%90%E5%A4%84%E7%90%86%E6%83%85%E5%86%B5%23&extparam=%23%E5%BE%90%E5%B7%9E%E9%80%9A%E6%8A%A5%E4%B8%B0%E5%8E%BF%E7%94%9F%E8%82%B2%E5%85%AB%E5%AD%A9%E5%A5%B3%E5%AD%90%E5%A4%84%E7%90%86%E6%83%85%E5%86%B5%23&luicode=10000011&lfid=100103type%3D1%26q%3D%23%E5%BE%90%E5%B7%9E%E9%80%9A%E6%8A%A5%E4%B8%B0%E5%8E%BF%E7%94%9F%E8%82%B2%E5%85%AB%E5%AD%A9%E5%A5%B3%E5%AD%90%E5%A4%84%E7%90%86%E6%83%85%E5%86%B5%23" data-hide=""><span class="surl-text">#徐州通报丰县生育八孩女子处理情况#</span></a> ',
  'user_id': 5520066736,
  'isLongText': False,
  'id': '4888190336372371',
  'user_screen_name': '一条猫昵',
  'user_verified': False,
  'u

In [16]:
long_text_url

27     http://m.weibo.cn/status/4878885122480570
73     http://m.weibo.cn/status/4867909669356826
98     http://m.weibo.cn/status/4865131916297485
140    http://m.weibo.cn/status/4857048234462019
142    http://m.weibo.cn/status/4843117188481443
147    http://m.weibo.cn/status/4837631676387397
148    http://m.weibo.cn/status/4837118595563697
149    http://m.weibo.cn/status/4836841842805211
150    http://m.weibo.cn/status/4835665516693741
153    http://m.weibo.cn/status/4832362468999496
162    http://m.weibo.cn/status/4799430642894067
164    http://m.weibo.cn/status/4798279873201987
169    http://m.weibo.cn/status/4791483976910694
170    http://m.weibo.cn/status/4791238903727172
187    http://m.weibo.cn/status/4778776205197601
195    http://m.weibo.cn/status/4770622107813831
199    http://m.weibo.cn/status/4764550190596361
202    http://m.weibo.cn/status/4756339447042053
207    http://m.weibo.cn/status/4753793642663641
213    http://m.weibo.cn/status/4752743863160713
235    http://m.weib

# 评论

In [29]:
id1 = df1[df1.comments_count!=0].id


1      4894409886011988
3      4894408934426148
5      4894206848142251
6      4894127475659678
7      4893856904251890
             ...       
526    4856847888294434
527    4856747960044476
528    4855412127700026
529    4853737615524421
530    4852612287169915
Name: id, Length: 232, dtype: object

In [8]:
df4 = pd.read_excel('./df4.xlsx')

In [19]:
dataf = df4
id1 = list(dataf[dataf.comments_count!=0].id)
data_ = []
for mid in id1:
    url_old = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id_type=0'''
    d = get_dict(url_old)
    print(d['ok'])
    if d['ok']==1:
        for j in range(len(d['data']['data'])): #第一页
            d1 = d['data']['data'][j]
            row = dict()
            row['comments_count']= d1['total_number']
            row['attitudes_count'] = d1['like_count']
            row['reposts_count'] = 0
            row['created_at']=d1['created_at']
            row['status_city']=0
            row['status_country']=0
            row['status_province']=d1['source']
            row['text']=d1['text']
            row['user_id']=d1['user']['id']
            row['id']=d1['id']
            row['isLongText']=False
            row['user_screen_name']=d1['user']['screen_name']
            row['user_verified']=d1['user']['verified']
            row['user_gender']=d1['user']['gender']
            row['user_follow_count']=d1['user']['follow_count']
            row['user_followers_count']=d1['user']['followers_count']
            if 'verified_reason' in d1['user'].keys():
                row['user_verified_reason']=d1['user']['verified_reason']
            else:
                row['user_verified_reason']=0
            data_.append(row)
        max_id_ = d['data']['max_id']
    while max_id_ != 0:
        print(max_id_)
        url_new = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id='''+str(max_id_)+'''&max_id_type=0'''
        d = get_dict(url_new)
        if d['ok'] == 1:
            for j in range(len(d['data']['data'])): #第n页
                d1 = d['data']['data'][j]
                row = dict()
                row['comments_count']= d1['total_number']
                row['attitudes_count'] = d1['like_count']
                row['reposts_count'] = 0
                row['created_at']=d1['created_at']
                row['status_city']=0
                row['status_country']=0
                row['status_province']=d1['source']
                row['text']=d1['text']
                row['user_id']=d1['user']['id']
                row['id']=d1['id']
                row['isLongText']=False
                row['user_screen_name']=d1['user']['screen_name']
                row['user_verified']=d1['user']['verified']
                row['user_gender']=d1['user']['gender']
                row['user_follow_count']=d1['user']['follow_count']
                row['user_followers_count']=d1['user']['followers_count']
                if 'verified_reason' in d1['user'].keys():
                    row['user_verified_reason']=d1['user']['verified_reason']
                else:
                    row['user_verified_reason']=0
                data_.append(row)
            max_id_ = d['data']['max_id']
        else:
             break

1
1
1
0
0
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
2
188140463655
2
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
2
140506902520180
2
139682264336438
2
138857812011382
2
138720197445934
2
138582758849948
2
138445324564632
2
138307904475050
2
138307890253274
2
138307882737339
2
138170630665625
2
138170554205557
2
138170504256216
2
138170482506804
2
138170452488995
2
4738501507547985
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

1
148066030872895
2
140506901954392
3
139544839373288
4
138857775126364
5
138720194191386
6
138582754463914
7
138445324274442
8
138307903614890
9
138307889872384
10
138307881144599
11
138170620144097
12
138170551471137
13
138170492665803
14
138170482340616
15
4740518296160796
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74


KeyboardInterrupt: 

0
1
2
3


In [36]:
data_

[{'comments_count': 0,
  'attitudes_count': 0,
  'reposts_count': 0,
  'created_at': 'Tue Apr 11 15:12:33 +0800 2023',
  'status_city': 0,
  'status_country': 0,
  'status_province': '来自北京',
  'text': '官冥日报',
  'user_id': 6677540108,
  'id': '4889357594397597',
  'isLongText': False,
  'user_screen_name': '_夤夜_',
  'user_verified': False,
  'user_gender': 'm',
  'user_follow_count': 143,
  'user_followers_count': '48',
  'user_verified_reason': 0},
 {'comments_count': 0,
  'attitudes_count': 8,
  'reposts_count': 0,
  'created_at': 'Fri Apr 07 16:43:15 +0800 2023',
  'status_city': 0,
  'status_country': 0,
  'status_province': '来自山东',
  'text': '真讽刺啊...这些人不收罚吗',
  'user_id': 5816021520,
  'id': '4887930873054202',
  'isLongText': False,
  'user_screen_name': 'HOSEASUGA',
  'user_verified': False,
  'user_gender': 'f',
  'user_follow_count': 821,
  'user_followers_count': '327',
  'user_verified_reason': 0},
 {'comments_count': 1,
  'attitudes_count': 3,
  'reposts_count': 0,
  'create

In [30]:
d['ok']

0

In [37]:
df4c = pd.DataFrame(data=data_)
text_list = list(df4c.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df4c1=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df4c = pd.concat([df4c,df4c1],axis=1)
user_followers_count_ = []

for i in list(df4c['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df4c['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df4c = df4c[col_name]
df4c.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id','id','isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

In [39]:
df4c.to_excel('./df4c.xlsx')

In [57]:
mid = '4888241075657822'
url_old = '''https://m.weibo.cn/comments/hotflow?id='''+mid+'''&mid='''+mid+'''&max_id_type=0'''
d = get_dict(url_old)

In [55]:
d1 = d['data']['data'][]
row = dict()
row['comments_count']= d1['total_number']
row['attitudes_count'] = d1['like_count']
row['reposts_count'] = 0
row['created_at']=d1['created_at']
row['status_city']=0
row['status_country']=0
row['status_province']=d1['source'].split('来自')[1]
row['text']=d1['text']
row['user_id']=d1['user']['id']
row['id']=d1['id']
row['isLongText']=False
row['user_screen_name']=d1['user']['screen_name']
row['user_verified']=d1['user']['verified']
row['user_gender']=d1['user']['gender']
row['user_follow_count']=d1['user']['follow_count']
row['user_followers_count']=d1['user']['followers_count']
if 'verified_reason' in d1['user'].keys():
    row['user_verified_reason']=d1['user']['verified_reason']
else:
    row['user_verified_reason']=0

206074574479037

In [58]:
d

{'ok': 1,
 'data': {'data': [{'created_at': 'Mon Apr 10 13:30:44 +0800 2023',
    'id': '4888969583528798',
    'rootid': '4888969583528798',
    'rootidstr': '4888969583528798',
    'floor_number': 5,
    'text': '这么有同理心的博主，就应该让家里的女孩子代代遇到这样的爹，成为这样的妈，生这样的娃。',
    'disable_reply': 0,
    'restrictOperate': 0,
    'source': '来自广东',
    'user': {'id': 1898785275,
     'screen_name': '开心元猫',
     'profile_image_url': 'https://tva2.sinaimg.cn/crop.0.0.512.512.180/712d29fbjw8ewnycsly4vj20e80e8dhi.jpg?KID=imgbed,tva&Expires=1682439904&ssig=Q7lKPAIpNr',
     'profile_url': 'https://m.weibo.cn/u/1898785275?uid=1898785275',
     'statuses_count': 21646,
     'verified': False,
     'verified_type': -1,
     'close_blue_v': False,
     'description': '喜欢读书、锻炼身体',
     'gender': 'f',
     'mbtype': 2,
     'svip': 0,
     'urank': 41,
     'mbrank': 1,
     'follow_me': False,
     'following': False,
     'follow_count': 2728,
     'followers_count': '607',
     'followers_count_str': '607',
    

In [48]:
[0]

{'created_at': 'Fri Apr 07 12:40:52 +0800 2023',
 'id': '4887869871098433',
 'rootid': '4887869871098433',
 'rootidstr': '4887869871098433',
 'floor_number': 5691,
 'text': '现在看还是气愤！！！',
 'disable_reply': 0,
 'restrictOperate': 0,
 'source': '来自山东',
 'user': {'id': 5765594002,
  'screen_name': '猫打呼噜我不打退堂鼓',
  'profile_image_url': 'https://tvax2.sinaimg.cn/crop.0.0.1080.1080.180/006ibQnEly8h7e8u3t03aj30u00u0q5j.jpg?KID=imgbed,tva&Expires=1682438488&ssig=sMt33luAnL',
  'profile_url': 'https://m.weibo.cn/u/5765594002?uid=5765594002',
  'statuses_count': 10276,
  'verified': False,
  'verified_type': -1,
  'close_blue_v': False,
  'description': '摆烂',
  'gender': 'f',
  'mbtype': 11,
  'svip': 0,
  'urank': 9,
  'mbrank': 2,
  'follow_me': False,
  'following': False,
  'follow_count': 519,
  'followers_count': '1042',
  'followers_count_str': '1042',
  'cover_image_phone': 'https://tva1.sinaimg.cn/crop.0.0.640.640.640/9d44112bjw1f1xl1c10tuj20hs0hs0tw.jpg',
  'avatar_hd': 'https://wx2.sina

In [51]:
df1.isLongText

0      False
1      False
2      False
3      False
4      False
       ...  
526    False
527     True
528     True
529    False
530     True
Name: isLongText, Length: 531, dtype: bool

In [17]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15',
           'accept': 'application/json, text/plain, */*',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'zh-CN,zh;q=0.9',
'cookie':'''_T_WM=34812304544; WEIBOCN_FROM=1110006030; __bid_n=187b8dbddb8fa1287a4207; FPTOKEN=VYXmO/jhda9+sgAHkMqaAKC+GmdAFsbxw5VJowWTFPS+mLi90XQlH/wv/7DhmN0OhnL4TnjmdUB39i+cqlUtjXY9SZqulq6XYtWHy6+xRZCvxiRCfblpvjpav2ydRO+IjYd43Pa+s+iOthIFAfkFQQlVuxk4CbgPKPGYLUz4YlEFaPJk9td0orCyDgmx0F8AWNkM7QlFPZDHkS5B6Jqx/+0WJsy/1L//40WdQlF0FJfvDLxyNIzdNQ0V93jE9eIoeT4e+UD80wQFNxoe9ikQ0rdWKrzOWj3jjKJ5vVOB0yQdvyHChXGNnwWm9Ew2zJyibO/0tqUaHnD5mQ51hxRqo4j4TixZ8/yGSOCsEaV76SgOxpOIik3X0CZlI9HdVioeA6pdIaAZbeuy6TKMO0ZS/g==|yhwBUYkYzSGi4VT73EqlqZxeB0HP7pQm1/Lh+KpLveA=|10|974c2ce98b1240c1681dfbb679ecaeeb; SUB=_2A25JQ5jyDeRhGeNP4lUV-CfMzzSIHXVqzzi6rDV6PUJbktAGLRH6kW1NTkU8YiiUbA-2uOuh7q6-XuNqwA5LiBnT; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5sXvVTPdl95v2mDK9FcLel5NHD95QfeK.NShn4ehBRWs4Dqc_xi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--4iKnNiKyhi--fi-i8i-zNi--fi-z7iK.X; SSOLoginState=1682434210; MLOGIN=1; XSRF-TOKEN=e8eef1; mweibo_short_token=c325ae1207; M_WEIBOCN_PARAMS=oid=4892900406593198&lfid=4892900406593198&luicode=20000174&uicode=20000061&fid=4892900406593198''',
'mweibo-pwa': '1',
'referer': 'https://m.weibo.cn/detail/4892900406593198',
'x-requested-with': 'XMLHttpRequest'}
url_new='https://m.weibo.cn/comments/hotflow?id=4892900406593198&mid=4892900406593198&max_id=139142112559718&max_id_type=0'
r=requests.get(url_new, headers = headers)

SyntaxError: invalid syntax (4139068488.py, line 1)

In [18]:
r.json()

{'ok': 0}

In [40]:
df3=pd.read_excel('./df3.xlsx')
dataf = df3
id1 = list(dataf[dataf.comments_count!=0].id)
data_ = []
for mid in id1:
    url_old = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id_type=0'''
    d = get_dict(url_old)
    print(d['ok'])
    if d['ok']==1:
        for j in range(len(d['data']['data'])): #第一页
            d1 = d['data']['data'][j]
            row = dict()
            row['comments_count']= d1['total_number']
            row['attitudes_count'] = d1['like_count']
            row['reposts_count'] = 0
            row['created_at']=d1['created_at']
            row['status_city']=0
            row['status_country']=0
            row['status_province']=d1['source']
            row['text']=d1['text']
            row['user_id']=d1['user']['id']
            row['id']=d1['id']
            row['isLongText']=False
            row['user_screen_name']=d1['user']['screen_name']
            row['user_verified']=d1['user']['verified']
            row['user_gender']=d1['user']['gender']
            row['user_follow_count']=d1['user']['follow_count']
            row['user_followers_count']=d1['user']['followers_count']
            if 'verified_reason' in d1['user'].keys():
                row['user_verified_reason']=d1['user']['verified_reason']
            else:
                row['user_verified_reason']=0
            data_.append(row)
        max_id_ = d['data']['max_id']
    while max_id_ != 0:
        print(max_id_)
        url_new = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id='''+str(max_id_)+'''&max_id_type=0'''
        d = get_dict(url_new)
        if d['ok'] == 1:
            for j in range(len(d['data']['data'])): #第n页
                d1 = d['data']['data'][j]
                row = dict()
                row['comments_count']= d1['total_number']
                row['attitudes_count'] = d1['like_count']
                row['reposts_count'] = 0
                row['created_at']=d1['created_at']
                row['status_city']=0
                row['status_country']=0
                row['status_province']=d1['source']
                row['text']=d1['text']
                row['user_id']=d1['user']['id']
                row['id']=d1['id']
                row['isLongText']=False
                row['user_screen_name']=d1['user']['screen_name']
                row['user_verified']=d1['user']['verified']
                row['user_gender']=d1['user']['gender']
                row['user_follow_count']=d1['user']['follow_count']
                row['user_followers_count']=d1['user']['followers_count']
                if 'verified_reason' in d1['user'].keys():
                    row['user_verified_reason']=d1['user']['verified_reason']
                else:
                    row['user_verified_reason']=0
                data_.append(row)
            max_id_ = d['data']['max_id']
        else:
             print('in the loop and break')
             break

1
139279242639848
191094642090
191022168062
191021073248
1
1
1
138179732918986
191023427755
0
1
1
1
1
1
1


In [47]:
df3c = pd.DataFrame(data=data_)
text_list = list(df3c.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df3c1=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df3c = pd.concat([df3c,df3c1],axis=1)
user_followers_count_ = []

for i in list(df3c['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df3c['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df3c = df3c[col_name]
df3c.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id','id','isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

In [ ]:
df3c.to_excel('./df3c.xlsx')

In [42]:
o=0
for i in range(0,100):
    print(i)
    while o<5:
        print('o:',o)
        o+=1
        if o>3:
            break

0
o: 0
o: 1
o: 2
o: 3
1
o: 4
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [5]:
df2=pd.read_excel('./df2.xlsx')
dataf = df2
id1 = list(dataf[dataf.comments_count!=0].id)
data_ = []
for mid in id1:
    url_old = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id_type=0'''
    d = get_dict(url_old)
    print(d['ok'])
    if d['ok']==1:
        for j in range(len(d['data']['data'])): #第一页
            d1 = d['data']['data'][j]
            row = dict()
            row['comments_count']= d1['total_number']
            row['attitudes_count'] = d1['like_count']
            row['reposts_count'] = 0
            row['created_at']=d1['created_at']
            row['status_city']=0
            row['status_country']=0
            row['status_province']=d1['source']
            row['text']=d1['text']
            row['user_id']=d1['user']['id']
            row['id']=d1['id']
            row['isLongText']=False
            row['user_screen_name']=d1['user']['screen_name']
            row['user_verified']=d1['user']['verified']
            row['user_gender']=d1['user']['gender']
            row['user_follow_count']=d1['user']['follow_count']
            row['user_followers_count']=d1['user']['followers_count']
            if 'verified_reason' in d1['user'].keys():
                row['user_verified_reason']=d1['user']['verified_reason']
            else:
                row['user_verified_reason']=0
            data_.append(row)
        max_id_ = d['data']['max_id']
    while max_id_ != 0:
        print(max_id_)
        url_new = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id='''+str(max_id_)+'''&max_id_type=0'''
        d = get_dict(url_new)
        if d['ok'] == 1:
            for j in range(len(d['data']['data'])): #第n页
                d1 = d['data']['data'][j]
                row = dict()
                row['comments_count']= d1['total_number']
                row['attitudes_count'] = d1['like_count']
                row['reposts_count'] = 0
                row['created_at']=d1['created_at']
                row['status_city']=0
                row['status_country']=0
                row['status_province']=d1['source']
                row['text']=d1['text']
                row['user_id']=d1['user']['id']
                row['id']=d1['id']
                row['isLongText']=False
                row['user_screen_name']=d1['user']['screen_name']
                row['user_verified']=d1['user']['verified']
                row['user_gender']=d1['user']['gender']
                row['user_follow_count']=d1['user']['follow_count']
                row['user_followers_count']=d1['user']['followers_count']
                if 'verified_reason' in d1['user'].keys():
                    row['user_verified_reason']=d1['user']['verified_reason']
                else:
                    row['user_verified_reason']=0
                data_.append(row)
            max_id_ = d['data']['max_id']
        else:
             print('in the loop and break')
             break

1
1
0
1
191114094841
0
1
0
1
1
191072278548
1
191083585066
1
1
1
0
1
84990859638221
191031118419
1
1
1
1
141753147738300
138866928056995
191041476424
191023817702
1
1
1
138866951480644
1
1
191087637969
1
1
1
1
141615776732246
138592111619815
138179793474412
0
1
1
1
1
206074574479037
191021402370
1
1
1
1
1
1
1
138179770377091
4888329407171414
in the loop and break
1
191031446660
in the loop and break
1
1
138866964076462
191055628632
191031160479
191024839173
in the loop and break
1
0
1
1
1
1
191046930834
1
1
1
1
1
1
0
1
139279251864279
138866937468670
191039709572
191027660959
0
1
1
1
82516963377196
0
0
1
1
1
1
191046833088
1
1
1
1
144639366559972
1
1
1
1
191025533798
1
1
1
1
1
142852659052361
138179861502820
191126637595
191032264832
191026156023
191024082312
in the loop and break
1
1
139279245594044
138179775381718
191042447400
191025374754
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
191105383583
1
1
191021431078
1
191040444365
1
1
1
144227050007523
138866931167861
191127656719
191026135298
1

In [8]:
df2c = pd.DataFrame(data=data_)
text_list = list(df2c.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df2c1=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df2c = pd.concat([df2c,df2c1],axis=1)
user_followers_count_ = []

for i in list(df2c['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df2c['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df2c = df2c[col_name]
df2c.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id','id','isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

/Users/chiajd/miniconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [9]:
df2c.to_excel('./df2c.xlsx')

In [10]:
df1=pd.read_excel('./df1.xlsx')
dataf = df1
id1 = list(dataf[dataf.comments_count!=0].id)
data_ = []
for mid in id1:
    url_old = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id_type=0'''
    d = get_dict(url_old)
    print(d['ok'])
    if d['ok']==1:
        for j in range(len(d['data']['data'])): #第一页
            d1 = d['data']['data'][j]
            row = dict()
            row['comments_count']= d1['total_number']
            row['attitudes_count'] = d1['like_count']
            row['reposts_count'] = 0
            row['created_at']=d1['created_at']
            row['status_city']=0
            row['status_country']=0
            row['status_province']=d1['source']
            row['text']=d1['text']
            row['user_id']=d1['user']['id']
            row['id']=d1['id']
            row['isLongText']=False
            row['user_screen_name']=d1['user']['screen_name']
            row['user_verified']=d1['user']['verified']
            row['user_gender']=d1['user']['gender']
            row['user_follow_count']=d1['user']['follow_count']
            row['user_followers_count']=d1['user']['followers_count']
            if 'verified_reason' in d1['user'].keys():
                row['user_verified_reason']=d1['user']['verified_reason']
            else:
                row['user_verified_reason']=0
            data_.append(row)
        max_id_ = d['data']['max_id']
    while max_id_ != 0:
        print(max_id_)
        url_new = '''https://m.weibo.cn/comments/hotflow?id='''+str(mid)+'''&mid='''+str(mid)+'''&max_id='''+str(max_id_)+'''&max_id_type=0'''
        d = get_dict(url_new)
        if d['ok'] == 1:
            for j in range(len(d['data']['data'])): #第n页
                d1 = d['data']['data'][j]
                row = dict()
                row['comments_count']= d1['total_number']
                row['attitudes_count'] = d1['like_count']
                row['reposts_count'] = 0
                row['created_at']=d1['created_at']
                row['status_city']=0
                row['status_country']=0
                row['status_province']=d1['source']
                row['text']=d1['text']
                row['user_id']=d1['user']['id']
                row['id']=d1['id']
                row['isLongText']=False
                row['user_screen_name']=d1['user']['screen_name']
                row['user_verified']=d1['user']['verified']
                row['user_gender']=d1['user']['gender']
                row['user_follow_count']=d1['user']['follow_count']
                row['user_followers_count']=d1['user']['followers_count']
                if 'verified_reason' in d1['user'].keys():
                    row['user_verified_reason']=d1['user']['verified_reason']
                else:
                    row['user_verified_reason']=0
                data_.append(row)
            max_id_ = d['data']['max_id']
        else:
             print('in the loop and break')
             break

1
1
1
1
1
1
1
1
139142112559718
82517263826242
191330765120
191330088280
1
83891651614456
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
191114262065
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
191054760542
in the loop and break
1
1
1
1
191111682895
1
191082503332
1
13881387964753461
1
1
1
1
0
1
1
1
191054689085
191029067701
1
191054689085
191029067701
1
1
1
1
1
191025159917
1
1
1
1
1
1
1
1
1
159345330341099
143127534755239
140241318962414
139554215848470
139279246909458
138867012090436
138866946010821
138866934442499
138866932485985
138866930718505
138866929866018
138866929083317
138866927988571
138866927608765
4891783252938774
in the loop and break
1
1
1
1
1
1
1
1
191074414060
1
191022307726
0
1
1
1
1
1
147525580695607
139966453452313
138317170824347
138179730961939
191061364359
191024505119
191022871994
191021965101
191021695107
191021492794
in the loop and break
1
1
1
1
1
1
0
1
138866927248488
138317175542276
191104547715
191021988818
191021503090
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
191043150439
1910224290

In [11]:
df1c = pd.DataFrame(data=data_)
text_list = list(df1c.text)
tags=[]
emojis=[]
texts = []
for txt in text_list:
    if len(re.findall('<a.*<\/a>',txt))!=0:
        bs=BeautifulSoup(txt)
        if len(bs.find_all('span',class_="surl-text")) !=0:
            tags_list = bs.find_all('span',class_="surl-text")
            tag = [tag.text for tag in tags_list]
            st=''
            for i in tag:
                st +=i
            tag = st
        else:
            tag = ''
        txt = re.sub('<a +href=.*?<\/span><\/a>','',txt)
    else:
        tag = ''
    tags.append(tag)
    bs_=BeautifulSoup(txt)
    if len(bs_.find_all('img'))==1:
        emoji = bs_.find_all('img')[0]
    elif len(bs_.find_all('img'))>1:
        emoji = [img['alt'] for img in bs_.find_all('img') if 'alt' in img.attrs.keys()]
        st = ''
        for i in emoji:
            st +=i
        emoji = st
    else:
        emoji = ''
    emojis.append(emoji)
    text = bs_.text
    if re.search('【 *】',text) != None:
        text = re.sub('【 *】','',text)
    texts.append(text)
emojis_str_list = [re.sub('\]','',re.sub('\[','',re.sub('\]\[',',',i)))  if len(i)!=0 else '' for i in emojis]
tags_str_list = []
for i in tags:
    if len(i)>0:
        sp = i.split('#')
        sP = [j for j in sp if len(j)>0]
        st = sP[0]
        for i in sP[1:]:
            st+= ','+i
        tags_str_list.append(st)
    else:
        tags_str_list.append(i)
df1c1=pd.DataFrame(data={'weibo_emoji':emojis_str_list,'after_texts':texts,'new_tags':tags_str_list})     
df1c = pd.concat([df1c,df1c1],axis=1)
user_followers_count_ = []

for i in list(df1c['user_followers_count']):
    if '万' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*10000))
    elif '亿' in i:
        user_followers_count_.append(int(float(re.search('\d+[\.]?\d*',i).group())*100000000))
    else:
        user_followers_count_.append(int(i))
df1c['user_followers_count']=user_followers_count_
col_name = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id', 'id' , 'isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'new_tags' , 'after_texts']
df1c = df1c[col_name]
df1c.columns = ['comments_count', 'attitudes_count', 'reposts_count', 'created_at', 'status_city', 'status_country', 'status_province', 'user_id','id','isLongText', 'user_screen_name', 'user_verified', 'user_gender', 'user_follow_count', 'user_followers_count', 'user_verified_reason', 'weibo_emoji', 'tags' , 'texts']

/Users/chiajd/miniconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [12]:
df1c.to_excel('./df1c.xlsx')

In [ ]:
df = pd.concat()